# Training Classifiers on Multiple UCI Datasets #

## Load cars.data ##

In [101]:
#grab the data from all the required datasets using csv.read (store csv file containing datasets)
using CSV, DataFrames

dfCar = CSV.read("car.data", DataFrame, header=0)

cols = ["buying","maint","doors","persons","lug_boot","safety", "classification"]
is_cat = [true,true,false,false,true,true,true]
is_cat_x = is_cat[1:6]
rename!(dfCar,cols)






,buying,maint,doors,persons,lug_boot,safety,classification
,String,String,String,String,String,String,String
1,vhigh,vhigh,2,2,small,low,unacc
2,vhigh,vhigh,2,2,small,med,unacc
3,vhigh,vhigh,2,2,small,high,unacc
4,vhigh,vhigh,2,2,med,low,unacc
5,vhigh,vhigh,2,2,med,med,unacc
6,vhigh,vhigh,2,2,med,high,unacc
7,vhigh,vhigh,2,2,big,low,unacc
8,vhigh,vhigh,2,2,big,med,unacc
9,vhigh,vhigh,2,2,big,high,unacc


In [102]:
#Categorical data for Bayes encode via label encoder,

using ScikitLearn
#first, replace 5more with 5, more with 5 in doors, persons (respectively)

dfCar[!,"doors"][dfCar[!,"doors"] .== "5more"] .= "5"
dfCar[!,"persons"][dfCar[!,"persons"] .== "more"] .= "5"

#dfCar

@sk_import preprocessing: LabelEncoder

#label encoder for categorical columns, could potentially use OHE 
label_encoder = LabelEncoder()

#X[:,3]
#dfCar[!,1]
#if categorical, 
for i in 1:7
    if(is_cat[i] == true)
        dfCar[!,i] = label_encoder.fit_transform(dfCar[!,i])
    end
end


X = Array(dfCar[:,1:6]) #feature matrix
y = Array(dfCar[:,7]) #labels 
    



1728-element Array{Int64,1}:
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 ⋮
 2
 1
 3
 2
 0
 1
 2
 1
 3
 2
 1
 3

### Train test partition for cars ###

In [103]:
using ScikitLearn

using ScikitLearn.CrossValidation: train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3) #70/30 split


4-element Array{Array,1}:
 Any[0 1 … 2 2; 0 0 … 0 1; … ; 1 2 … 2 0; 2 0 … 2 0]
 Any[1 2 … 2 2; 0 0 … 1 1; … ; 3 3 … 1 2; 1 3 … 1 0]
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  1, 2, 1, 2, 0, 0, 2, 0, 1, 0]
 [0, 2, 2, 0, 2, 3, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 0, 3, 2, 0]

## Bayesian Classifier for Cars ##

In [106]:
@sk_import naive_bayes: GaussianNB
@sk_import naive_bayes: CategoricalNB
@sk_import metrics: accuracy_score
using PyCall
@pyimport joblib as jl
gnb = CategoricalNB()

@time fit!(gnb, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(gnb,X_train, y_train))

println("Accuracy on Test Data: ")
println(score(gnb,X_test, y_test))

#time to test model on test data
@time gnb.predict(X_test)


jl.dump(gnb,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")






#slightly underfitted model

  0.014328 seconds (16.95 k allocations: 265.344 KiB)
Accuracy on Training Data: 
0.8684863523573201
Accuracy on Test Data: 
0.8805394990366089
  0.004388 seconds (7.30 k allocations: 119.141 KiB)
Size of model: 
2955 bytes


## Classification Trees for Cars ##

In [105]:
@sk_import tree: DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5)
@time fit!(dt, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(dt, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(dt, X_test, y_test))


#time to test model on test data
@time dt.predict(X_test)



jl.dump(dt,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")



#slightly overfitted

  0.007899 seconds (16.95 k allocations: 265.344 KiB)
Accuracy on Training Data: 
0.8610421836228288
Accuracy on Test Data: 
0.8535645472061657
  0.003618 seconds (7.30 k allocations: 119.141 KiB)
Size of model: 
3381 bytes


## SVM for Cars ##

In [38]:
@sk_import svm: SVC

svc = SVC(gamma=0.5, C=0.5)
@time fit!(svc, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(svc, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(svc, X_test, y_test))

#time to test model on test data
@time svc.predict(X_test)

jl.dump(svc,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

#slightly overfitting!

  0.075393 seconds (16.95 k allocations: 265.344 KiB)
Accuracy on Training Data: 


0.9660876757650951
Accuracy on Test Data: 
0.928709055876686
  0.045793 seconds (7.30 k allocations: 119.141 KiB)
Size of model: 
86512 bytes


## Neural Networks for Cars ##

In [40]:
@sk_import neural_network: MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=600)
@time fit!(clf, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(clf, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(clf, X_test, y_test))

#time to test model on test data
@time clf.predict(X_test)

jl.dump(clf,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")


  2.229638 seconds (16.95 k allocations: 265.344 KiB)
Accuracy on Training Data: 
0.9511993382961125
Accuracy on Test Data: 
0.9344894026974951
  0.007012 seconds (7.30 k allocations: 119.141 KiB)
Size of model: 
48969 bytes


C:\Users\darkr\.julia\conda\3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\darkr\.julia\conda\3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\darkr\.julia\conda\3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric va

## Load abalone.data ##

In [118]:
#grab the data from all the required datasets using csv.read (store csv file containing datasets)

dfAb = CSV.read("abalone.data", DataFrame, header=0)

#column titles
cols = ["sex", "length", "diameter", "height", "whole weight", "shucked weight", 
    "viscera weight", "shell weight", "rings"]
rename!(dfAb,cols)



,sex,length,diameter,height,whole weight,shucked weight,viscera weight
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,M,0.455,0.365,0.095,0.514,0.2245,0.101
2,M,0.35,0.265,0.09,0.2255,0.0995,0.0485
3,F,0.53,0.42,0.135,0.677,0.2565,0.1415
4,M,0.44,0.365,0.125,0.516,0.2155,0.114
5,I,0.33,0.255,0.08,0.205,0.0895,0.0395
6,I,0.425,0.3,0.095,0.3515,0.141,0.0775
7,F,0.53,0.415,0.15,0.7775,0.237,0.1415
8,F,0.545,0.425,0.125,0.768,0.294,0.1495
9,M,0.475,0.37,0.125,0.5095,0.2165,0.1125


## Data Cleaning and Preprocessing for Abalone ##

In [119]:
#predicting rings, want to normalize
#no missing values, want to encode sex 
#onehot encode only sex
@sk_import preprocessing:OneHotEncoder
@sk_import preprocessing:LabelEncoder
@sk_import compose:ColumnTransformer

#split age into 3 classes (1-8, 9-10, 11-29)
for i in 1:size(dfAb,1)
    if(dfAb[i,9]<=8 && dfAb[i,9]>=1)
        dfAb[i,9] = 1
    elseif (dfAb[i,9]<=10 && dfAb[i,9]>=9)
        dfAb[i,9] = 2
    elseif (dfAb[i,9]<=29 && dfAb[i,9]>=11)
        dfAb[i,9] = 3
    end
end


le = LabelEncoder()
columnTransformer = ColumnTransformer([("encoder", 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder="passthrough") 

X = Array(dfAb[:,Not(9)])
y = Array(dfAb[:,9])
#print(y)

X[:,1] = le.fit_transform(X[:,1])

#enc = OneHotEncoder(categories=["sex"],sparse=false)





4177-element Array{Int64,1}:
 2
 2
 0
 2
 1
 1
 0
 0
 2
 0
 0
 2
 2
 ⋮
 1
 1
 2
 0
 2
 2
 2
 0
 2
 2
 0
 2

### Train Test split for Abalone ###

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

#train test split -> exists in python but is it in julia

4-element Array{Array,1}:
 Real[2 0.535 … 0.1595 0.2155; 0 0.585 … 0.2325 0.358; … ; 2 0.625 … 0.2045 0.25; 0 0.595 … 0.1905 0.289]
 Real[1 0.5 … 0.116 0.17; 1 0.56 … 0.209 0.275; … ; 0 0.635 … 0.292 0.35; 1 0.55 … 0.1495 0.221]
 [1, 3, 2, 3, 2, 1, 1, 1, 1, 2  …  1, 1, 3, 1, 1, 3, 2, 2, 1, 2]
 [2, 3, 3, 3, 2, 1, 3, 3, 1, 3  …  3, 2, 1, 3, 1, 1, 2, 2, 2, 2]

## Bayes Classifier for Abalone ##

In [117]:
@sk_import naive_bayes: GaussianNB

gnb = GaussianNB()
@time fit!(gnb, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(gnb,X_train, y_train))

println("Accuracy on Test Data: ")
println(score(gnb,X_test, y_test))

#time to test model on test data
@time gnb.predict(X_test)

jl.dump(gnb,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  0.012722 seconds (52.63 k allocations: 822.969 KiB)
Accuracy on Training Data: 
0.5805679096818337
Accuracy on Test Data: 
0.5877192982456141
  0.005464 seconds (22.61 k allocations: 364.047 KiB)
Size of model: 
1101 bytes


## Decision Tree Classifier for Abalone ##

In [47]:
@sk_import tree: DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5)
@time fit!(dt, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(dt, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(dt, X_test, y_test))

#time to test model on test data
@time dt.predict(X_test)

jl.dump(dt,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")


  0.016884 seconds (52.63 k allocations: 822.969 KiB)
Accuracy on Training Data: 
0.667464933287718
Accuracy on Test Data: 
0.6291866028708134
  0.005089 seconds (22.61 k allocations: 364.047 KiB)
Size of model: 
6053 bytes


## SVM Classifier for Abalone ##

In [48]:
@sk_import svm: SVC

svc = SVC(gamma=0.5, C=1)
@time fit!(svc, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(svc, X_train, y_train))

println("Accuracy on Test Data: ")
println(score( svc, X_test, y_test))


#time to test model on test data
@time svc.predict(X_test)

jl.dump(svc,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  0.334587 seconds (52.63 k allocations: 822.969 KiB)
Accuracy on Training Data: 


0.630516592541909
Accuracy on Test Data: 
0.6275917065390749
  0.263678 seconds (22.61 k allocations: 364.047 KiB)
Size of model: 
222837 bytes


## Neural Network for Abalone ##

In [49]:
@sk_import neural_network: MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=700)
@time fit!(clf, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(clf, X_train, y_train))

println("Accuracy on Test Data: ")
println(score(clf, X_test, y_test))

#time to test model on test data
@time clf.predict(X_test)

jl.dump(clf,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  2.639389 seconds (52.63 k allocations: 822.969 KiB)
Accuracy on Training Data: 
0.6616489907629148
Accuracy on Test Data: 
0.6722488038277512
  0.007193 seconds (22.61 k allocations: 364.047 KiB)
Size of model: 
48074 bytes


## Load Madelon Data

In [121]:
#No need for train test split, data given as test, train and validation
trainDF = CSV.read("madelon_train.data", DataFrame, header=0)
testDF = CSV.read("madelon_test.data", DataFrame, header=0)
valDF = CSV.read("madelon_valid.data", DataFrame, header=0)

train_labels = CSV.read("madelon_train.labels", DataFrame, header=0)
val_labels = CSV.read("madelon_valid.labels", DataFrame, header=0)

#use val as test, test_y not given


,Column1
,Int64
1,-1
2,-1
3,-1
4,1
5,-1
6,1
7,-1
8,-1
9,-1


## Data Cleaning and Preprocessing for Madelon ##

In [122]:
#have label vector (binary classification)
#have feature matrix (need to remove last row of missings, we have labels already)
X_train = Array(select(trainDF, Not(:Column501)))
y_train = Array(train_labels[:,1])
#print(X_train)
X_test = Array(select(valDF, Not(:Column501)))
y_test = Array(val_labels[:,1])

#normalizing doesnt actually make a difference, performance remains the same (all
#features are weighted equally)
# we do norm here but dont actually use it - difference is negligible
using LinearAlgebra
X_train_norm=normalize(X_train)
X_test_norm=normalize(X_test)
#min max normalize features???
# using StatsBase
# X_train_norm = StatsBase.fit(ZScoreTransform, X_train)
# StatsBase.transform(X_train_norm, X_train)

600×500 Array{Float64,2}:
 0.00180331  0.00169503  0.00191531  …  0.00185931  0.00195265  0.00190411
 0.00181077  0.00189665  0.00184064     0.00184811  0.00204599  0.00178091
 0.00180331  0.00194518  0.00189291     0.0017361   0.00189665  0.00187798
 0.00176971  0.00188171  0.00215053     0.00174357  0.00206092  0.00193025
 0.00184811  0.00176971  0.00195265     0.00184064  0.00188918  0.00183318
 0.00176971  0.00189291  0.00188171  …  0.00172117  0.00198625  0.00194145
 0.00179584  0.00177344  0.00190038     0.0017361   0.00184438  0.00193772
 0.00181451  0.00166143  0.0016577      0.00176971  0.00166517  0.00179584
 0.00179957  0.00180331  0.00196012     0.00182944  0.00177717  0.00173984
 0.00181824  0.00184438  0.00185931     0.00179211  0.00188545  0.00197505
 0.00179584  0.00191531  0.00191531  …  0.00182197  0.00189665  0.00174357
 0.00179211  0.00181824  0.00191531     0.00184064  0.00193772  0.00194892
 0.00175477  0.00162783  0.00168757     0.00176597  0.00182571  0.00178091

## Bayes Classifier for Madelon ##

In [125]:
@sk_import naive_bayes: GaussianNB

gnb = GaussianNB()
@time fit!(gnb, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(gnb,X_train, y_train))

println("Accuracy on Testing Data: ")
println( score(gnb,X_test, y_test))

#time to test model on test data
@time gnb.predict(X_test)

jl.dump(gnb,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  0.014216 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 
0.714
Accuracy on Testing Data: 
0.5916666666666667
  0.004627 seconds (36 allocations: 6.500 KiB)
Size of model: 
16696 bytes


## Decision Tree Classifier for Madelon ##

In [126]:
@sk_import tree: DecisionTreeClassifier

#limit depth to lower overfitting of tree (prune)
dt = DecisionTreeClassifier(max_depth=2)
@time fit!(dt, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(dt, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(dt, X_test, y_test))

#time to test model on test data
@time dt.predict(X_test)


jl.dump(dt,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")


  0.081047 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 
0.651
Accuracy on Test Data: 
0.665
  0.000923 seconds (36 allocations: 6.500 KiB)
Size of model: 
1673 bytes


### SVM Classifier for Madelon ###

In [127]:
@sk_import svm: LinearSVC

svc = LinearSVC(max_iter=3000)
@time fit!(svc, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(svc, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(svc, X_test, y_test))

#time to test model on test data
@time svc.predict(X_test)

jl.dump(svc,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  9.905345 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 
0.56
Accuracy on Test Data: 
0.5183333333333333
  0.000988 seconds (36 allocations: 6.500 KiB)
Size of model: 
4745 bytes


C:\Users\darkr\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### Neural Network Classifier for Madelon ###

In [58]:
@sk_import neural_network: MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=500)
@time fit!(clf, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(clf, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(clf, X_test, y_test))

#time to test model on test data
@time clf.predict(X_test)

jl.dump(clf,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")


  1.115958 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 
0.547
Accuracy on Test Data: 
0.565
  0.002618 seconds (36 allocations: 6.500 KiB)
Size of model: 
1612019 bytes


## Loading KDD Dataset ##

In [92]:
dtK = CSV.read("kddcup.data_10_percent_corrected", DataFrame, header=0)

#label vector will come from names
dtNames = CSV.read("kddcup.names", DataFrame, header=1)

┌ Warning: thread = 1 warning: only found 1 / 23 columns around data row: 2. Filling remaining columns with `missing`
└ @ CSV C:\Users\darkr\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1 / 23 columns around data row: 3. Filling remaining columns with `missing`
└ @ CSV C:\Users\darkr\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1 / 23 columns around data row: 4. Filling remaining columns with `missing`
└ @ CSV C:\Users\darkr\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1 / 23 columns around data row: 5. Filling remaining columns with `missing`
└ @ CSV C:\Users\darkr\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1 / 23 columns around data row: 6. Filling remaining columns with `missing`
└ @ CSV C:\Users\darkr\.julia\packages\CSV\YUbbG\src\file.jl:603
┌ Warning: thread = 1 warning: only found 1 / 23 columns around data row: 7. Filling 

,back,buffer_overflow,ftp_write,guess_passwd,imap
,String,Missing,Missing,Missing,Missing
1,duration: continuous.,missing,missing,missing,missing
2,protocol_type: symbolic.,missing,missing,missing,missing
3,service: symbolic.,missing,missing,missing,missing
4,flag: symbolic.,missing,missing,missing,missing
5,src_bytes: continuous.,missing,missing,missing,missing
6,dst_bytes: continuous.,missing,missing,missing,missing
7,land: symbolic.,missing,missing,missing,missing
8,wrong_fragment: continuous.,missing,missing,missing,missing
9,urgent: continuous.,missing,missing,missing,missing


## Cleaning KDD Data ##

In [8]:
cols = Array(dtNames[:,1])

#clean titles for columns (pretty dataframe)
for i in 1:size(cols,1)
    cols[i]=replace(cols[i],": continuous."=>"")
    cols[i]=replace(cols[i],": symbolic."=>"")
    #print(cols[i,:])
end

#add target column (label we are predicting)
push!(cols,"target")

#set column titles to cols
rename!(dtK,cols)

dtK[:,"target"]



494021-element PooledArrays.PooledArray{String,UInt32,1,Array{UInt32,1}}:
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 ⋮
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."
 "normal."

## Data Preprocessing for KDD ##

In [60]:
# for i in 1:size(dtK,2)
#     println(i)
#     println
#     println(describe(dtK[:,i]))
# end

X = dtK[:,1:41]
#binary classification, attack or not attack (if normal, not attack, if not, attack)
#0: not an attack (normal)
#1: attack (not normal)
y = Vector{Int64}()

#turn into binary classifciation, attack or not attack
for i in 1:size(dtK,1)
    if dtK[i,42] == "normal."
        push!(y,0)
    else
        push!(y,1)
    end
end


In [61]:
# want to label encode features 2,3,4

le = LabelEncoder()

#print(y)
# for i in 1:size(X,1)
#     print(typeof(X[i,2]))
# end
X[!,2] = le.fit_transform(X[!,2])
X[!,3] = le.fit_transform(X[!,3])
X[!,4] = le.fit_transform(X[!,4])

#enc = OneHotEncoder(categories=["sex"],sparse=false)
#OHE crashes computer 
X = Array(X)
y = Array(y)


494021-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

## KDD Train Test Split ##

In [62]:
using ScikitLearn.CrossValidation: train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

4-element Array{Array,1}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0]
 [0.0 2.0 … 0.0 0.0; 0.0 1.0 … 1.0 1.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0]
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 0, 1, 0]
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 0, 0, 1, 0, 1, 1]

## Bayesian NB Classifier for KDD Dataset ##

In [63]:
@sk_import naive_bayes: GaussianNB

gnb = GaussianNB()
@time fit!(gnb, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(gnb,X_train, y_train))

println("Accuracy on Test Data: ")
println( score(gnb,X_test, y_test))

#time to train model on test data
@time gnb.predict(X_test)

jl.dump(gnb,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  0.266101 seconds (35.37 k allocations: 1.882 MiB)
Accuracy on Training Data: 


0.9798099556408937
Accuracy on Test Data: 
0.9798659982322022
  0.098137 seconds (25.01 k allocations: 2.474 MiB)
Size of model: 
2005 bytes


## Decision Tree Classifier for KDD ##

In [64]:
@sk_import tree: DecisionTreeClassifier

#limit depth to lower overfitting of tree (prune)
dt = DecisionTreeClassifier(max_depth=2)
@time fit!(dt, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(dt, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(dt, X_test, y_test))

#time to train model on test data
@time dt.predict(X_test)

jl.dump(dt,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

  0.324659 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 
0.9868917973245733
Accuracy on Test Data: 
0.9865998232202258


  0.015627 seconds (37 allocations: 1.132 MiB)
Size of model: 
1669 bytes


## SVM Classifier for KDD ##

In [65]:
@sk_import svm: LinearSVC

svc = LinearSVC(max_iter=250)
@time fit!(svc, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(svc, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(svc, X_test, y_test))

#time to train model on test data
@time svc.predict(X_test)

jl.dump(svc,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")

 14.306521 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 
0.9904081384790668
Accuracy on Test Data: 
0.9901759026226832
  0.010161 seconds (37 allocations: 1.132 MiB)
Size of model: 
1070 bytes


C:\Users\darkr\.julia\conda\3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


## Neural Network Classifier for KDD ##

In [66]:
@sk_import neural_network: MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=200)
@time fit!(clf, X_train, y_train)

println("Accuracy on Training Data: ")
println(score(clf, X_train, y_train))

println("Accuracy on Test Data: ")
println( score(clf, X_test, y_test))

#time to train model on test data
@time clf.predict(X_test)

jl.dump(clf,"model.pkl")
println("Size of model: ")
println(filesize("model.pkl")," bytes")


 51.398093 seconds (22 allocations: 1.125 KiB)
Accuracy on Training Data: 


0.9973020178477447
Accuracy on Test Data: 
0.9971998623546796
  0.154773 seconds (37 allocations: 1.132 MiB)
Size of model: 
143117 bytes
